In [5]:
# !pip install minicons

  Using cached minicons-0.2.14-py3-none-any.whl (22 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


# Import and Helper functions

In [6]:
# -- fix path --
from pathlib import Path
import sys
sys.path.append(str(Path('..').resolve()))
from source.resources import *
from source.metrics import *
from source.helper import *
from source.preprocessor import *
import Levenshtein

from nltk import word_tokenize
from functools import lru_cache
import unicodedata
from collections import Counter
# from nltk import word_tokenize
import nltk
import magic
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.corpus import stopwords
# import spacy
# nlp = spacy.load("en_core_web_sm")

from joblib import Memory
current_dir = Path('.')

DATASETS = [Dataset.BenchLS, Dataset.LexMTurk, Dataset.NNSeval, Dataset.SemEval2012, Dataset.TSAR_EN]


In [7]:
def __normalize_string(text):
        text = text.replace('``', '"')
        text = text.replace('`', "'")
        text = text.replace("''", '"')
        text = text.strip('"')

        return text 
    


In [8]:
from minicons import cwe
model = cwe.CWE('bert-base-uncased', device='cpu')

In [9]:
# Example 1

context_words = [("I went to the bank to withdraw money.", "bank"), 
                 ("i was at the bank of the river ganga!", "bank")]


print(model.extract_representation(context_words, layer = 12))

tensor([[ 0.5737, -0.5295, -0.0294,  ..., -0.7193, -0.3905, -0.1534],
        [-0.8258, -0.4308,  0.2744,  ..., -0.5987, -0.6984,  0.2087]])


In [10]:
from minicons import scorer

mlm_model = scorer.MaskedLMScorer('bert-base-uncased', 'cpu')
ilm_model = scorer.IncrementalLMScorer('distilgpt2', 'cpu')
s2s_model = scorer.Seq2SeqScorer('t5-base', 'cpu')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using pad_token, but it is not set yet.
/Users/kim/.pyenv/versions/3.9.4/envs/ConLS/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Using bos_token, but it is not set yet.


In [16]:
stimuli = ["The keys to the cabinet are on the table.",
           "The keys to the cabinet is on the table."]
print(ilm_model.sequence_score(stimuli, reduction = lambda x: -x.sum(0).item()))

print(mlm_model.sequence_score(stimuli, reduction = lambda x: -x.sum(0).item()))
print(s2s_model.sequence_score(stimuli, source_format = 'blank'))
print(s2s_model.sequence_score(stimuli, source_format = 'copy'))

[39.8797492980957, 42.75846862792969]
[13.962686538696289, 23.415119171142578]
[-7.910909175872803, -7.835635185241699]
[-10.555521965026855, -9.532543182373047]


### BenchLS

In [27]:
#BenchLS
data = load_data(Dataset.BenchLS)
data

,text,complex_word,complex_word_index,candidates
0,"in march 1992 , linux version 0.95 was the fir...",pieces,35,"[[""parts""], [""bits""], [""components""], [""inform..."
1,much of the water carried by these streams is ...,diverted,9,"[[""redirected""], [""rerouted""], [""changed"", ""mo..."
2,harry also becomes the worthy possessor of the...,possessor,5,"[[""owner""], [""holder""], [""keeper""], [""buyer"", ..."
3,"escapologists escape from handcuffs , straitja...",perils,24,"[[""dangers""], [""hazards"", ""situations"", ""dange..."
4,"the storm continued , crossing the outer banks...",retained,13,"[[""kept""], [""held""], [""maintained""], [""regaine..."
...,...,...,...,...
924,because of the obvious incapacity of these bou...,function,14,"[[""job""], [""role""], [""task"", ""purpose""], [""wor..."
925,"the composition , structure , function and dis...",function,5,"[[""role""], [""purpose""], [""mapping"", ""working""]]"
926,"however , their arguments are identical in fun...",function,7,"[[""role"", ""purpose""], [""activity""], [""performa..."
927,its main function is to act as a bridge betwee...,function,2,"[[""job""], [""task"", ""role"", ""purpose""]]"


In [49]:
for i in range(len(data[:20])):
    print(f'{i}. ', '='*80)
    
    item = data.iloc[i]
    # print(item)
    text = item['text']
    complex_word = item['complex_word']
    candidates = json.loads(item['candidates'])
    candidates = [item for items in candidates for item in items] # flatten array
    # score = mlm_model.sequence_score(stimuli, reduction = lambda x: -x.sum(0).item())
    print('orig: ', text)
    print(candidates)
    # print(candidates)
    print('>', f'{complex_word:<15}', mlm_model.sequence_score([text], reduction = lambda x: -x.sum(0).item())[0])
    for num, candidate in enumerate(candidates):  
        replaced_text = text.replace(complex_word, candidate)
        print(f'{num}. {candidate:<15}', mlm_model.sequence_score([replaced_text], reduction = lambda x: -x.sum(0).item())[0])
    # break

0.  ================================================================================
orig:  in march 1992 , linux version 0.95 was the first to be capable of running x . this large version number jump was due to a feeling that a version 1.0 with no major missing pieces was imminent .
['parts', 'bits', 'components', 'information', 'elements', 'items', 'component', 'part', 'sections']
> pieces          73.05708169937134
0. parts           72.30447816848755
1. bits            73.99261045455933
2. components      76.35317897796631
3. information     75.30068874359131
4. elements        75.57960414886475
5. items           78.03495121002197
6. component       81.94555711746216
7. part            79.75300073623657
8. sections        76.00263595581055
1.  ================================================================================
orig:  much of the water carried by these streams is diverted .
['redirected', 'rerouted', 'changed', 'moved', 'separated', 'turned', 'altered', 'split', 'switc

### LexMTurk

In [51]:
data = load_data(Dataset.LexMTurk)
data

,text,complex_word,complex_word_index,candidates
0,"in march 1992 , linux version 0.95 was the fir...",pieces,34,"[[""parts""], [""bits""], [""components""], [""compon..."
1,much of the water carried by these streams is ...,diverted,9,"[[""redirected""], [""rerouted""], [""changed"", ""mo..."
2,harry also becomes the worthy possessor of the...,possessor,5,"[[""owner""], [""holder""], [""keeper""], [""buyer"", ..."
3,"escapologists escape from handcuffs , straitja...",perils,24,"[[""dangers""], [""difficulties"", ""danger"", ""rest..."
4,"the storm continued , crossing the outer banks...",retained,13,"[[""kept""], [""held""], [""maintained""], [""regaine..."
...,...,...,...,...
495,the marriage was marked from the outset by sex...,outset,6,"[[""beginning""], [""start""], [""begining""], [""beg..."
496,"born as alfons karg , he was trained as a tele...",profession,19,"[[""job""], [""career""], [""work"", ""job."", ""specia..."
497,"some features , however , are reminiscent of t...",reminiscent,6,"[[""similar""], [""remindful""], [""reminders""], [""..."
498,the shinto pantheon alone consists of an uncou...,consists,4,"[[""is made""], [""is made up""], [""is""], [""contai..."


In [53]:
for i in range(len(data[:20])):
    print(f'{i}. ', '='*80)
    
    item = data.iloc[i]
    # print(item)
    text = item['text']
    complex_word = item['complex_word']
    candidates = json.loads(item['candidates'])
    candidates = [item for items in candidates for item in items] # flatten array
    # score = mlm_model.sequence_score(stimuli, reduction = lambda x: -x.sum(0).item())
    print('orig: ', text)
    print(candidates)
    # print(candidates)
    print('>', f'{complex_word:<15}', mlm_model.sequence_score([text], reduction = lambda x: -x.sum(0).item())[0])
    for num, candidate in enumerate(candidates):  
        replaced_text = text.replace(complex_word, candidate)
        print(f'{num}. {candidate:<15}', mlm_model.sequence_score([replaced_text], reduction = lambda x: -x.sum(0).item())[0])
    # break

0.  ================================================================================
orig:  in march 1992 , linux version 0.95 was the first to be capable of running x. this large version number jump was due to a feeling that a version 1.0 with no major missing pieces was imminent .
['parts', 'bits', 'components', 'component', 'sections', 'elements', 'part', 'information', 'items']
> pieces          73.05708169937134
0. parts           72.30447816848755
1. bits            73.99261045455933
2. components      76.35317897796631
3. component       81.94555711746216
4. sections        76.00263595581055
5. elements        75.57960414886475
6. part            79.75300073623657
7. information     75.30068874359131
8. items           78.03495121002197
1.  ================================================================================
orig:  much of the water carried by these streams is diverted .
['redirected', 'rerouted', 'changed', 'moved', 'drawn away', 'turned', 'separated', 'switched', '

In [55]:
from minicons import cwe

model = cwe.CWE('bert-base-uncased')

In [63]:
# cosine similarity
from sklearn.metrics.pairwise import cosine_similarity 

for i in range(len(data[:10])):
    print(f'{i}. ', '='*80)
    
    item = data.iloc[i]
    # print(item)
    text = item['text']
    complex_word = item['complex_word']
    candidates = json.loads(item['candidates'])
    candidates = [item for items in candidates for item in items] # flatten array
    
    print('orig: ', text)
    print(candidates)
    # print(candidates)
    
    orig_embeddings = model.extract_representation([(text, complex_word)], layer = 12)
    # print(orig_embeddings)
    for num, candidate in enumerate(candidates):  
        replaced_text = text.replace(complex_word, candidate)
        embeddings = model.extract_representation([(replaced_text, candidate)], layer = 12)
        print(f'{complex_word} {candidate:<15}', cosine_similarity(orig_embeddings, embeddings)[0])
    # break

0.  ================================================================================
orig:  in march 1992 , linux version 0.95 was the first to be capable of running x. this large version number jump was due to a feeling that a version 1.0 with no major missing pieces was imminent .
['parts', 'bits', 'components', 'component', 'sections', 'elements', 'part', 'information', 'items']
pieces parts           [0.8410455]
pieces bits            [0.7285559]
pieces components      [0.7789433]
pieces component       [0.7239192]
pieces sections        [0.82848614]
pieces elements        [0.82115823]
pieces part            [0.7312142]
pieces information     [0.71391153]
pieces items           [0.8332304]
1.  ================================================================================
orig:  much of the water carried by these streams is diverted .
['redirected', 'rerouted', 'changed', 'moved', 'drawn away', 'turned', 'separated', 'switched', 'split', 'altered', 'led away', 'sent away', 'veered

### NNSeval

In [ ]:
data = load_data(Dataset.NNSeval)
data

### TSAR EN

In [ ]:
data = load_data(Dataset.TSAR_EN)
data

# Check frequency

In [ ]:
data = load_data(Dataset.LexMTurk, return_frequency=True)

items = []
for row in data.iloc:
    candidates = row['candidates']
    if len(candidates) > 1 and candidates[0][1] == candidates[1][1]:
        print(candidates[:5])
        items.append(items)
print(Dataset.LexMTurk, len(data), len(items)) 


In [ ]:

data = load_data(Dataset.NNSeval, return_frequency=True)

items = []
for row in data.iloc:
    candidates = row['candidates']
    if len(candidates) > 1 and candidates[0][1] == candidates[1][1]:
        print(candidates[:5])
        items.append(items)
print(Dataset.NNSeval, len(data), len(items)) 

In [ ]:

data = load_data(Dataset.BenchLS, return_frequency=True)

items = []
for row in data.iloc:
    candidates = row['candidates']
    if len(candidates) > 1 and candidates[0][1] == candidates[1][1]:
        print(candidates[:5])
        items.append(items)
print(Dataset.BenchLS, len(data), len(items)) 

# Database similarity check


In [ ]:
memory = Memory(current_dir / 'cache', verbose=False)

@memory.cache()
def sentence_similarity(sentence, sentences):
    scores = []
    for s in sentences:
        score = Levenshtein.ratio(sentence, s)
        scores.append((s, score))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return scores[0]

def compare_two_datasets(dataset1, dataset2):
    # use Levenshtein similarity to check
    data1 = load_data(dataset1)
    data2 = load_data(dataset2)
    # benchls
    sentences = [row['text'].lower() for row in data2.iloc]
    similarity_scores = []
    for i in tqdm(range(len(data1))):
        row = data1.iloc[i]
        sent = row['text'].lower()
        scores = sentence_similarity(sent, sentences)
        similarity_scores.append((sent, scores))    

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1][1], reverse=True)
    nb_similar_sent = 0
    for sent, scores in similarity_scores:
        sent2, score = scores
        if score > 0.9:
            nb_similar_sent +=1
        if score > 0.5:
            print(f'\n{sent}')
            print(sent2, score)
    print(f'Compare {dataset1} and {dataset2}')
    print(f'Total Count of similar sentences: {nb_similar_sent}')


## BenchLS and LexMTurk

In [ ]:
compare_two_datasets(Dataset.BenchLS, Dataset.LexMTurk)

## BenchLS and NNSeval

In [ ]:
compare_two_datasets(Dataset.BenchLS, Dataset.NNSeval)

## BenchLS and TSAR_EN

In [ ]:
compare_two_datasets(Dataset.BenchLS, Dataset.TSAR_EN)

## BenchLS and SemEval2012

In [ ]:
compare_two_datasets(Dataset.BenchLS, Dataset.SemEval2012)

In [ ]:
compare_two_datasets(Dataset.NNSeval, Dataset.SemEval2012)

## TSAR EN

In [ ]:
compare_two_datasets(Dataset.TSAR_EN, Dataset.BenchLS)

In [ ]:
compare_two_datasets(Dataset.TSAR_EN, Dataset.NNSeval)

In [ ]:
compare_two_datasets(Dataset.TSAR_EN, Dataset.LexMTurk)

# Features Analysis
---

## CharRatioFeature

In [ ]:
def char_ratio_check(dataset, i_candidate=0, print_words=False):
    print(dataset, i_candidate)
    
    data = load_data(dataset)
    size = len(data)
    count_bigger = 0 
    count_smaller = 0
    equal = 0
    length_dict = {}
    for row in data.iloc:
        complex = row['complex_word']
        candidates = row['candidates']
        
        candidate = candidates[i_candidate] if len(candidates) > i_candidate else candidates[0]
        
        # print(complex, candidates)
        if print_words:
            print(f'{complex:15} {candidate:15} {len(complex):15} {len(candidate):15}')
        if len(complex) > len(candidate):
            count_bigger += 1
        elif len(complex) < len(candidate):
            count_smaller += 1
        else:
            equal += 1 
    print(f'Complex is longer than candidate: {count_bigger / size * 100 :.2f}%')
    print(f'Complex is shorter than candidate: {count_smaller / size * 100 :.2f}%')
    print(f'Complex is equal to candidate: {equal / size * 100 :.2f}%')



In [ ]:
for dataset in DATASETS:
    char_ratio_check(dataset, i_candidate=0)
    char_ratio_check(dataset, i_candidate=1)
    char_ratio_check(dataset, i_candidate=2)
    char_ratio_check(dataset, i_candidate=3)
    char_ratio_check(dataset, i_candidate=4)
    print()

## WordRank

In [ ]:

def word_rank_check(dataset, i_candidate=0, verbose=False):
    data = load_data(dataset)
    word_rank = WordRank()
    count = 0
    for row in data.iloc:
        complex_word = row['complex_word']
        candidates = row['candidates']
        # candidate = candidates[0]
        candidate = candidates[i_candidate] if len(candidates) > i_candidate else candidates[0]
        # if candidate == complex_word and len(candidates) > 1: candidate = candidates[1]
        
        score_complex = word_rank.get_lexical_complexity_score(complex_word)
        score_candidate = word_rank.get_lexical_complexity_score(candidate)
        ratio = score_candidate / score_complex
        if ratio < 1: count += 1
        # if ratio == 1: print(complex_word, candidate)
        if verbose:
            print(f'{complex_word:10} {candidate:15} {score_complex:.2f} {score_candidate:.2f} {ratio:.2f}') 
        
    print(f'{dataset} {i_candidate:15} : {count / len(data) * 100:.2f}')
    # print()

In [ ]:
for dataset in DATASETS:
    word_rank_check(dataset)
    word_rank_check(dataset, i_candidate=1)
    word_rank_check(dataset, i_candidate=2)
    word_rank_check(dataset, i_candidate=3)
    word_rank_check(dataset, i_candidate=4)
    
# SemEval 2012 includes many first candidates the same as the complex words.

## WordSyllableRatioFeature

In [ ]:

def word_syllables_check(dataset, verbose=False):
    data = load_data(dataset)
    feature = WordSyllable()
    count = 0
    for row in data.iloc:
        complex_word = row['complex_word']
        candidates = row['candidates']
        candidate = candidates[0]
        # if candidate == complex_word and len(candidates) > 1: candidate = candidates[1]
        
        nb_syllable_complex = feature.count_syllable(complex_word)
        nb_syllable_candidate = feature.count_syllable(candidate)
        ratio = safe_division(nb_syllable_candidate, nb_syllable_complex)
        # ratio = safe_division(nb_syllable_complex, nb_syllable_candidate)
        
        if ratio < 1: count += 1
        # if ratio == 1: print(complex_word, candidate)
        
        if verbose:
            print(f'{complex_word:10} {candidate:15} {nb_syllable_complex:.2f} {nb_syllable_candidate:.2f} {ratio:.2f}') 
        
    print(f'{dataset:12}: {count / len(data) * 100:.2f}')
    print()

In [ ]:
for dataset in DATASETS:
    word_syllables_check(dataset)

## CandidateRankingRatioFeature

In [ ]:

def candidate_ranking_check(dataset, verbose=False):
    data = load_data(dataset)
    feature = CandidateRanking()
    count = 0
    for row in data.iloc:
        complex_word = row['complex_word']
        candidates = row['candidates']
        candidate = candidates[0]
        # if candidate == complex_word and len(candidates) > 1: candidate = candidates[1]
        
        # score_complex = feature.count_syllable(complex_word)
        # score_candidate = feature.count_syllable(candidate)
        # ratio = safe_division(score_candidate, score_complex)
        ratio = feature.get_ranking_ratio()
        
        if ratio < 1: count += 1
        # if ratio == 1: print(complex_word, candidate)
        
        if verbose:
            print(f'{complex_word:10} {candidate:15} {score_complex:.2f} {score_candidate:.2f} {ratio:.2f}') 
        
    print(f'{dataset:12}: {count / len(data) * 100:.2f}')
    print()